In [12]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr # oh yeah!
import pandas as pd
import io
# imports
import os
import glob
from dotenv import load_dotenv
import gradio as gr
# imports for langchain, plotly and Chroma

from langchain.document_loaders import DirectoryLoader, TextLoader,  PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyAt


In [14]:
openai = OpenAI()


google.generativeai.configure()

In [30]:
MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [18]:
folders = glob.glob("pdf/*.pdf")

documents = []

for folder in folders:
    doc_type = os.path.splitext(os.path.basename(folder))[0]

    # Load all PDFs inside this folder
    loader = PyPDFLoader(folder)
    folder_docs = loader.load()

    # Attach folder name as metadata
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

print(f"Loaded {len(documents)} documents")

Loaded 210 documents


In [25]:
# Read in documents using LangChain's loaders
# Take everything in all the sub-folders of our knowledgebase

folders = glob.glob("knowledge-base/*")

# With thanks to CG and Jon R, students on the course, for this fix needed for some users 
text_loader_kwargs = {'encoding': 'utf-8'}
# If that doesn't work, some Windows users might need to uncomment the next line instead
# text_loader_kwargs={'autodetect_encoding': True}

#documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [26]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1088, which is longer than the specified 1000


In [28]:
print(f"Loaded {len(documents)} documents")

Loaded 241 documents


In [27]:
print (chunks[100])

page_content='89
IIMA HR Policy Manual 2023
4. Financial upgradation under the scheme shall be allowed in the immediate next higher level 
pay in the hierarchy of revised levels as per the policy of the Institute.
5. Financial upgradation would be on non-functional basis (i.e. Group D employee is categorised 
as Group D only) subject to eligibility and within the Group. 
6. As such there shall be no additional financial upgradation for the senior employee on the 
ground that the junior employee in the level has got higher pay under the Scheme. If the 
senior employee remains on LWP or/and does not enhance the education qualification, then 
possible that a junior employee may get an advantage compared to a senior employee.
7. Following are the existing Level change in the Institute for Group D employees:
Sr. Level Type of 
Promotion Minimum service in the Institute (from DOJ)
1 1 -- Entry
2 2 SP #1 10 years
3 3 SP #2 20 years
4 4 SR #1 27 years
5 5 SR #2 33/35 years (33 years to be cons

In [31]:
embeddings = OpenAIEmbeddings()

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 333 documents


In [32]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

The vectors have 1,536 dimensions


In [33]:
# Prework

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
doc_types = [metadata['doc_type'] for metadata in result['metadatas']]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

ValueError: 'Aditya_Solanki_Cloud' is not in list

In [ ]:
prompts = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Can you please create a test Dataset for me for Service now Incidents with few open and closed incidents and count ot be 10000 records"}
  ]
system_message = "You are a helpful assistant"

In [ ]:
def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    csv_string =  completion.choices[0].message.content
    print (csv_string)
    return csv_string

In [ ]:
def process_and_display_csv(prompt):
        csv_data = message_gpt(prompt)
        # Convert the CSV string to a DataFrame for better display or further processing
        try:
            df = pd.read_csv(io.StringIO(csv_data))
            return df
        except Exception as e:
            return f"Error processing CSV: {e}\nRaw CSV data:\n{csv_data}"

iface = gr.Interface(
    fn=process_and_display_csv,
    inputs=gr.Textbox(lines=5, label="Enter your prompt for CSV generation:"),
    outputs=gr.Dataframe(label="Generated CSV Data"), # Use gr.Dataframe to display tabular data
    title="GPT CSV Generator"
)

iface.launch(share=False) # Set share=True to get a public link if needed

In [ ]:
#gr.Interface(fn=message_gpt, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

In [ ]:
import gradio as gr
import openai
import pandas as pd
import numpy as np
import io
import random
import tempfile
from datetime import timedelta

In [ ]:
def chatbot_fn(message, history):
    history = history or []

    # special command: create dataset
    if "create csv" in message.lower():
        df = generate_servicenow_data(10000)
        csv_path = "servicenow_dataset.csv"
        df.to_csv(csv_path, index=False)
        history.append((message, (csv_path, "📂 Download ServiceNow CSV (10k rows)")))
        return history, ""  # clear input

    # otherwise, ask GPT
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",  # or "gpt-4o", adjust if needed
        messages=[{"role": "system", "content": "You are a helpful ServiceNow assistant."}] +
                 [{"role": "user", "content": m[0]} for m in history if isinstance(m[0], str)] +
                 [{"role": "assistant", "content": m[1]} for m in history if isinstance(m[1], str)] +
                 [{"role": "user", "content": message}]
    )
    reply = completion.choices[0].message.content
    history.append((message, reply))
    return history, ""  # clear input

In [ ]:
def generate_servicenow_data(n=10000):
    np.random.seed(42)
    random.seed(42)

    priorities = ["Low", "Medium", "High", "Critical"]
    impacts = ["Low", "Medium", "High"]
    urgencies = ["Low", "Medium", "High"]
    statuses = ["Open", "In Progress", "Resolved", "Closed"]
    categories = ["Network", "Software", "Hardware", "Database", "Security"]
    subcategories = {
        "Network": ["VPN", "LAN", "WAN"],
        "Software": ["Email", "OS", "Application"],
        "Hardware": ["Laptop", "Desktop", "Printer"],
        "Database": ["Oracle", "MySQL", "SQL Server"],
        "Security": ["Phishing", "Malware", "Access"],
    }
    resolution_codes = ["Solved Remotely", "Solved via On-Site", "Workaround Provided", "Not Reproducible"]

   # Base dates
    opened_dates = pd.date_range("2025-07-01", periods=n, freq="h")

    # Random statuses
    status_selected = np.random.choice(statuses, n, p=[0.3, 0.3, 0.2, 0.2])  # 60% still open/in progress

    closed_dates = []
    resolution_codes_selected = []
    resolved_by = []

    for i, status in enumerate(status_selected):
        if status in ["Resolved", "Closed"]:
            closed_dt = opened_dates[i] + timedelta(hours=random.randint(1, 72))
            closed_dates.append(closed_dt)
            resolution_codes_selected.append(random.choice(resolution_codes))
            resolved_by.append(f"user{random.randint(401,600)}")
        else:
            closed_dates.append(pd.NaT)  # no close date
            resolution_codes_selected.append(None)
            resolved_by.append(None)

    categories_selected = np.random.choice(categories, n)
    subcategories_selected = [random.choice(subcategories[cat]) for cat in categories_selected]

    data = {
        "Incident_ID": [f"INC{i:06d}" for i in range(1, n+1)],
        "Opened_At": opened_dates,
        "Closed_At": closed_dates,
        "Opened_By": [f"user{random.randint(1,200)}" for _ in range(n)],
        "Assigned_To": [f"user{random.randint(201,400)}" for _ in range(n)],
        "Priority": np.random.choice(priorities, n),
        "Impact": np.random.choice(impacts, n),
        "Urgency": np.random.choice(urgencies, n),
        "Category": categories_selected,
        "Subcategory": subcategories_selected,
        "Status": status_selected,
        "Resolution_Code": resolution_codes_selected,
        "Resolved_By": resolved_by,
        "Short_Description": [f"Issue {i} - {random.choice(categories)} related" for i in range(1, n+1)],
        "Description": [f"Detailed description of incident {i}, auto-generated for testing." for i in range(1, n+1)],
        "Updated_At": [d + timedelta(hours=random.randint(1,100)) for d in opened_dates],
    }

    return pd.DataFrame(data)
def chatbot_fn(message, history):
    history = history or []

    if "create csv" in message.lower():
        df = generate_servicenow_data(10000)
        tmp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
        csv_path = "servicenow_dataset.csv"
        df.to_csv(tmp_file.name, index=False)
    
        # reply: preview + file download
        preview = df.head().to_markdown()
        bot_reply = f"Here’s a preview of the dataset:\n\n{preview}"

        history.append((message, bot_reply))
        history.append(("", (csv_path, "📂 Download ServiceNow CSV (10k rows)")))

        return history, ""  # clear input


    # otherwise, ask GPT
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",  # or "gpt-4o", adjust if needed
        messages=[{"role": "system", "content": "You are a helpful ServiceNow assistant."}] +
                 [{"role": "user", "content": m[0]} for m in history if isinstance(m[0], str)] +
                 [{"role": "assistant", "content": m[1]} for m in history if isinstance(m[1], str)] +
                 [{"role": "user", "content": message}]
    )
    reply = completion.choices[0].message.content
    history.append((message, reply))
    return history, ""  # clear input


# Gradio UI
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="ServiceNow Chatbot", height=500)
    msg = gr.Textbox(
        label="Type your message here",
        placeholder="Ask me something or type 'create csv'...",
    )

    msg.submit(chatbot_fn, [msg, chatbot], [chatbot, msg])

demo.launch()